# Episode 17 — Exploring Operator Classes and Their Applications

Qiskit v2.0 focuses operator work primarily in `qiskit.quantum_info`. In this lesson you’ll learn when to use:

- **`SparsePauliOp`**: sparse linear combinations of Pauli strings (great for observables and many physics Hamiltonians).
- **`Pauli`**: single Pauli string with an optional global phase.
- **`Operator`**: dense linear operators (best for a *few* qubits or when you really need the matrix).

We’ll build operators, manipulate them algebraically, convert among types, and compute basic expectation values.

In [3]:
import numpy as np
from qiskit.quantum_info import Statevector
from qiskit.quantum_info.operators import Operator, Pauli, SparsePauliOp

## 1) Building `SparsePauliOp`

`SparsePauliOp` encodes a sum of Pauli strings. Use it for observables/Hamiltonians—fast, memory-friendly, and composable.

In [4]:
# 3-qubit example: H = 0.7 * Z(0)Z(1)  -  0.4i * X(1)X(2)  +  0.2 * Y(0)
# We place substrings on target indices; unspecified qubits become I.
H = SparsePauliOp.from_sparse_list([
    ("ZZ", [0, 1], 0.7),
    ("XX", [1, 2], -0.4j),
    ("Y",  [0],    0.2),
], num_qubits=3)
print(H)

SparsePauliOp(['IZZ', 'XXI', 'IIY'],
              coeffs=[0.7+0.j , 0. -0.4j, 0.2+0.j ])


### Algebra with `SparsePauliOp`
- Addition and scalar multiplication
- Composition (matrix product) and tensor product
- Convert to a dense `Operator` when needed

In [5]:
G = SparsePauliOp.from_sparse_list([
    ("Z", [2], 1.0),
    ("XY", [0, 1], -0.25)
], num_qubits=3)

print("H + G:\n", H + G)
print("\n2 * H:\n", 2 * H)

print("\nComposition H @ G (same qubits):\n", H @ G)

print("\nTensor: H ⊗ Z (extra ancilla qubit)\n")
Z1 = SparsePauliOp.from_sparse_list([("Z", [0], 1.0)], num_qubits=1)
print(H.tensor(Z1))

# Dense view (for small systems):
H_dense = H.to_operator()
print("\nDense matrix shape:", H_dense.data.shape)

H + G:
 SparsePauliOp(['IZZ', 'XXI', 'IIY', 'ZII', 'IYX'],
              coeffs=[ 0.7 +0.j ,  0.  -0.4j,  0.2 +0.j ,  1.  +0.j , -0.25+0.j ])

2 * H:
 SparsePauliOp(['IZZ', 'XXI', 'IIY'],
              coeffs=[1.4+0.j , 0. -0.8j, 0.4+0.j ])

Composition H @ G (same qubits):
 SparsePauliOp(['ZZZ', 'IXY', 'YXI', 'XZX', 'ZIY', 'IYZ'],
              coeffs=[ 0.7  +0.j  , -0.175+0.j  , -0.4  +0.j  , -0.1  +0.j  ,  0.2  +0.j  ,
  0.   +0.05j])

Tensor: H ⊗ Z (extra ancilla qubit)

SparsePauliOp(['IZZZ', 'XXIZ', 'IIYZ'],
              coeffs=[0.7+0.j , 0. -0.4j, 0.2+0.j ])

Dense matrix shape: (8, 8)


## 2) Expectation values with `SparsePauliOp`
Use a statevector for noiseless demos. Convert `SparsePauliOp` to `Operator` on demand, then evaluate.

In [6]:
# |psi> = |+ + 0> for 3 qubits
psi = Statevector.from_label("0++")  # Qiskit reads left->right: qubit2, qubit1, qubit0
exp_val = psi.expectation_value(H.to_operator())
print("⟨H⟩ on |0++> =", complex(exp_val))

⟨H⟩ on |0++> = 0j


## 3) `Pauli` strings
`Pauli` is a single string with optional global phase. Handy for low-level checks or quick matrices.

In [7]:
p = Pauli("-iYZ")  # 2-qubit operator with a -i phase on Y⊗Z
print("pauli:", p)
print("dim:", p.dim)
print("phase (0:+1,1:+i,2:-1,3:-i):", p.phase)
print("matrix:\n", p.to_matrix())

# Adjoint and simple composition
p_dag = p.adjoint()
print("\nAdjoint equals inverse (unitary):", np.allclose(p.to_matrix() @ p_dag.to_matrix(), np.eye(4)))

# Convert a single Pauli to SparsePauliOp and back
p_op = SparsePauliOp.from_list([(str(p), 1.0)])
print("\nAs SparsePauliOp:", p_op)

pauli: -iYZ
dim: (4, 4)
phase (0:+1,1:+i,2:-1,3:-i): 1
matrix:
 [[ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.-0.j]
 [ 1.-0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]]

Adjoint equals inverse (unitary): True

As SparsePauliOp: SparsePauliOp(['YZ'],
              coeffs=[0.-1.j])


## 4) Dense `Operator`
Use `Operator` when you already have (or truly need) a dense matrix. Keep in mind the exponential cost.

In [8]:
# Build a 2-qubit dense operator and inspect dims
mat = np.array([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
], dtype=complex)
O = Operator(mat)
print(O)
print("data shape:", O.data.shape)
print("input_dims:", O.input_dims(), "output_dims:", O.output_dims())

# Type conversions
O_as_sparse = SparsePauliOp.from_operator(O)
O_roundtrip = Operator(O_as_sparse)
print("\nRoundtrip equal:", np.allclose(O.data, O_roundtrip.data))

Operator([[0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
          [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
          [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
data shape: (4, 4)
input_dims: (2, 2) output_dims: (2, 2)

Roundtrip equal: True


## 5) Mini‑exercise — Build an Ising‑ZZ Hamiltonian (3 qubits)
**Task:** Construct 
$H = \sum_{i=0}^{1} Z_i Z_{i+1} + 0.3\,X_1$ as a `SparsePauliOp` and compute $\langle H \rangle$ on $|+ + +\rangle$.

Hint: use `from_sparse_list` with appropriate qubit indices. Then convert to `Operator` for the expectation value with a `Statevector`.

In [ ]:
# Your attempt here
from math import sqrt

# TODO: build H_ising and compute exp_val_ising
pass

<details>
<summary><b>Show solution</b></summary>

```python
H_ising = SparsePauliOp.from_sparse_list([
    ("ZZ", [0, 1], 1.0),
    ("ZZ", [1, 2], 1.0),
    ("X",  [1],    0.3),
], num_qubits=3)

psi_plus = Statevector.from_label("+++")
exp_val_ising = psi_plus.expectation_value(H_ising.to_operator())
exp_val_ising
```

</details>

## 6) Sanity check — two constructions, one operator
Verify that building a simple $XX$ term via `Pauli` vs. `SparsePauliOp` yields the same dense matrix.

In [11]:
xx_sparse = SparsePauliOp.from_sparse_list([("XX", [0,1], 1.0)], num_qubits=2)
xx_pauli  = Pauli("XX")
eq = np.allclose(xx_sparse.to_operator().data, Operator(xx_pauli).data)
print("Equivalent dense operators:", eq)

Equivalent dense operators: True


### Key takeaways
- Prefer **`SparsePauliOp`** for observables and Hamiltonians.
- Use **`Pauli`** for single strings, low-level algebra, or generating small test matrices.
- Fall back to **`Operator`** when you truly need dense matrices (few qubits).

## Additional information

**Created by:** Ricard Santiago Raigada García

**Version:** 1.0.0